<a href="https://colab.research.google.com/github/JuniorTorresMTJ/TowerBank/blob/main/notebooks/TowerBank_Limpeza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***
***
#TowerBank
***
***

<img src="https://raw.githubusercontent.com/JuniorTorresMTJ/TowerBank/main/img/TowerBankLog.png" alt="win" width="800"/>

***
#OBJETIVO
***

Criação de uma maquina preditiva que aprove ou negue crédito do banco digital TowerBank.


***
#IMPORTAÇÃO DAS BIBLIOTECAS
***

In [1]:
import pandas as pd
import numpy as np
from google.colab import files

import warnings
warnings.filterwarnings('ignore')

***
#DADOS
***

Base inspirada  e adaptada de um desafio do [Kaggle](https://www.kaggle.com/rikdifos/credit-card-approval-prediction)

Para baixar as bases adaptadas e a base limpa acesse meu [Github](https://github.com/JuniorTorresMTJ/TowerBank/tree/main/dados)

In [2]:
df_clientes_cadastrados = pd.read_csv('https://raw.githubusercontent.com/JuniorTorresMTJ/TowerBank/main/dados/clientes_cadastrados.csv?raw=True')
df_clientes_aprovados   = pd.read_csv('https://raw.githubusercontent.com/JuniorTorresMTJ/TowerBank/main/dados/clientes_aprovados.csv?raw=True')


***
#LIMPEZA DA BASE DE CLIENTES CADASTRADOS
***


Primeiramente limpar a base que possui as informações dos clientes cadastrados.

Vamos verificar quais são as colunas e informações da base utilizando o `.head()`, para visualizar mais linhas apenas precisamos colocar o número de linhas que queremos dentro dos parenteses, caso não colocamos nada de parâmetro irá aparecer 5 primeiras linhas.

In [3]:
df_clientes_cadastrados.head()

,ID_Cliente,Genero,Tem_Carro,Tem_Casa_Propria,Qtd_Filhos,Rendimento_Anual,Categoria_de_renda,Grau_Escolaridade,Estado_Civil,Moradia,Idade,Tem_Celular,Tem_telefone_trabalho,Tem_telefone_fixo,Tem_email,Ocupacao,Tamanho_Familia,Anos_empregado
0,5008804,H,1,1,0,427500.0,Empregado,Ensino superior,Uniao estavel,Alugado,33.0,1,1,0,0,Outros,2.0,12.4
1,5008805,H,1,1,0,427500.0,Empregado,Ensino superior,Uniao estavel,Alugado,33.0,1,1,0,0,Outros,2.0,12.4
2,5008806,H,1,1,0,112500.0,Empregado,Ensino medio,Casado,Casa propria,59.0,1,0,0,0,Seguranca,2.0,3.1
3,5008808,M,0,1,0,270000.0,Associado comercial,Ensino medio,Solteiro,Casa propria,52.0,1,0,1,1,Venda,1.0,8.4
4,5008809,M,0,1,0,270000.0,Associado comercial,Ensino medio,Solteiro,Casa propria,52.0,1,0,1,1,Venda,1.0,8.4


Para verificar quantas colunas e quantas colunas temos, apenas precisamos usar o `.shape` que é escrito da seguinte forma: `(linhas, Colunas)`.

Então temos 438.557 linhas e 18 colunas.


In [4]:
df_clientes_cadastrados.shape

(438557, 18)

Vamos averiguar se cada linha é um cliente diferente. Como podemos ver abaixo, tem algums clientes que aparecem em duas linhas diferentes.


In [5]:
df_clientes_cadastrados['ID_Cliente'].value_counts()

7603224    2
7742853    2
7836711    2
7091721    2
7022327    2
          ..
5372264    1
6412652    1
6410605    1
6367602    1
6291456    1
Name: ID_Cliente, Length: 438510, dtype: int64

Olhando quantos temos de duplicados por ID utilizando o `.duplicated()` e o parâmatro `keep = False`, que significa que ele vai pegar todos os duplicados.

In [6]:
df_clientes_cadastrados[df_clientes_cadastrados['ID_Cliente'].duplicated(keep=False)].shape

(94, 18)

Veja que os IDs duplicados o genero sempre muda, sempre um é M e outro é F


In [7]:
df_clientes_cadastrados[df_clientes_cadastrados['ID_Cliente'].duplicated(keep=False)].sort_values(by='ID_Cliente')

,ID_Cliente,Genero,Tem_Carro,Tem_Casa_Propria,Qtd_Filhos,Rendimento_Anual,Categoria_de_renda,Grau_Escolaridade,Estado_Civil,Moradia,Idade,Tem_Celular,Tem_telefone_trabalho,Tem_telefone_fixo,Tem_email,Ocupacao,Tamanho_Familia,Anos_empregado
426818,7022197,H,1,1,3,135000.0,Empregado,Ensino medio,Casado,Casa propria,33.0,1,0,0,1,Construcao Civil,5.0,2.0
425023,7022197,M,0,1,0,450000.0,Associado comercial,Ensino superior,Separado,Casa propria,54.0,1,0,0,1,Outros,1.0,4.9
431545,7022327,M,0,1,0,135000.0,Associado comercial,Ensino medio,Solteiro,Casa propria,40.0,1,0,0,0,Alta tecnologia,1.0,14.5
431911,7022327,H,1,1,0,256500.0,Associado comercial,Ensino superior,Casado,Casa propria,59.0,1,0,0,1,Equipe principal,2.0,4.6
425486,7023108,H,1,1,1,67500.0,Empregado,Ensino medio,Casado,Casa propria,42.0,1,1,0,0,Equipe principal,3.0,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426563,7836711,M,0,1,2,292500.0,Empregado,Ensino superior,Casado,Casa propria,38.0,1,0,1,0,Contabilidade,4.0,12.0
421464,7836971,H,1,0,1,157500.0,Empregado,Ensino medio,Casado,Casa propria,38.0,1,0,0,0,Outros,3.0,15.1
428620,7836971,M,0,1,0,103500.0,Empregado,Ensino medio,Uniao estavel,Casa propria,37.0,1,0,1,0,Venda,2.0,7.7
422068,7838075,H,0,1,0,337500.0,Associado comercial,Ensino medio,Casado,Casa propria,50.0,1,0,0,1,Motorista,2.0,3.5


Então vamos criar uma variável só com os IDs repetidos.

In [8]:
id_repetidos = df_clientes_cadastrados[df_clientes_cadastrados['ID_Cliente'].duplicated(keep=False)]['ID_Cliente']

Agora vamos remover os IDs repetidos e verificar se tem algum ID com a soma igual a 2.

In [9]:
df_clientes_cadastrados_unicos = df_clientes_cadastrados.drop(id_repetidos.index)

In [10]:
df_clientes_cadastrados_unicos['ID_Cliente'].value_counts()

6293503    1
5166449    1
6210931    1
5174645    1
5170551    1
          ..
7410037    1
6357367    1
6375806    1
5683584    1
6291456    1
Name: ID_Cliente, Length: 438463, dtype: int64

Vamos utilizar um `.info()` para entender quais são os tipo de **dados das colunas** e se a coluna possui `valores nulos`.

In [11]:
df_clientes_cadastrados_unicos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438463 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID_Cliente             438463 non-null  int64  
 1   Genero                 438463 non-null  object 
 2   Tem_Carro              438463 non-null  int64  
 3   Tem_Casa_Propria       438463 non-null  int64  
 4   Qtd_Filhos             438463 non-null  int64  
 5   Rendimento_Anual       438463 non-null  float64
 6   Categoria_de_renda     438463 non-null  object 
 7   Grau_Escolaridade      438463 non-null  object 
 8   Estado_Civil           438463 non-null  object 
 9   Moradia                438463 non-null  object 
 10  Idade                  438463 non-null  float64
 11  Tem_Celular            438463 non-null  int64  
 12  Tem_telefone_trabalho  438463 non-null  int64  
 13  Tem_telefone_fixo      438463 non-null  int64  
 14  Tem_email              438463 non-nu

Para verificar se possui outliers e as estatísticas básicas das colunas utilizamos o `.describe()`.

In [12]:
df_clientes_cadastrados_unicos.describe()

,ID_Cliente,Tem_Carro,Tem_Casa_Propria,Qtd_Filhos,Rendimento_Anual,Idade,Tem_Celular,Tem_telefone_trabalho,Tem_telefone_fixo,Tem_email,Tamanho_Familia,Anos_empregado
count,4.384630e+05,438463.000000,438463.000000,438463.000000,4.384630e+05,438463.000000,438463.0,438463.000000,438463.000000,438463.000000,438463.000000,438463.000000
mean,6.021894e+06,0.371906,0.693352,0.427384,1.875220e+05,43.834032,1.0,0.206129,0.287762,0.108196,2.194461,-165.936036
std,5.713554e+05,0.483314,0.461103,0.724879,1.100832e+05,11.472815,0.0,0.404525,0.452720,0.310628,0.897194,380.201131
min,5.008804e+06,0.000000,0.000000,0.000000,2.610000e+04,21.000000,1.0,0.000000,0.000000,0.000000,1.000000,-1000.700000
25%,5.609340e+06,0.000000,0.000000,0.000000,1.215000e+05,34.000000,1.0,0.000000,0.000000,0.000000,2.000000,1.000000
50%,6.047690e+06,0.000000,1.000000,0.000000,1.611000e+05,43.000000,1.0,0.000000,0.000000,0.000000,2.000000,4.000000
75%,6.454118e+06,1.000000,1.000000,1.000000,2.250000e+05,53.000000,1.0,0.000000,1.000000,0.000000,3.000000,8.500000
max,7.999952e+06,1.000000,1.000000,19.000000,6.750000e+06,69.000000,1.0,1.000000,1.000000,1.000000,20.000000,48.000000


Pegando só a coluna Anos_empregado, podemos verificar que olhando para o min, podemos ver que está um númeor gritante e ninguem trabalha -1000 anos, então precisamos tratar essa coluna.

In [13]:
df_clientes_cadastrados_unicos['Anos_empregado'].describe()

count    438463.000000
mean       -165.936036
std         380.201131
min       -1000.700000
25%           1.000000
50%           4.000000
75%           8.500000
max          48.000000
Name: Anos_empregado, dtype: float64

Vamos criar um `.query() ` olha a distribuição da coluna e veja que a grande **maioria é pensionista**, então não podemos remover todos que estão menor ou igual a zero.

In [14]:
df_clientes_cadastrados_unicos.query("Anos_empregado <=0")[['Anos_empregado', "Categoria_de_renda"]].value_counts()

Anos_empregado  Categoria_de_renda 
-1000.7         Pensionista            75314
 0.0            Empregado                 26
                Servidor publico          11
                Associado comercial        7
dtype: int64

Então para não afetar a quantidade de dados importante vamos fazer uma substituição dos **-1000.7 para -1** utilizando o `.replace()` e verificar a melhora.

In [15]:
df_clientes_cadastrados_unicos['Anos_empregado'] = df_clientes_cadastrados_unicos['Anos_empregado'].replace(-1000.7, -1)
df_clientes_cadastrados_unicos['Anos_empregado'].describe()

count    438463.000000
mean          5.780633
std           6.733151
min          -1.000000
25%           1.000000
50%           4.000000
75%           8.500000
max          48.000000
Name: Anos_empregado, dtype: float64

Agora vamos remover os outliers do **Rendimento Anual**, para isso vamos remover todos que são **maiores e menores** que a soma da média mais duas vezes o desvio padrão e mostrar quantos estão nessa condição.

In [16]:
coluna = df_clientes_cadastrados_unicos['Rendimento_Anual']

coluna_med = coluna.mean()
coluna_std = coluna.std()

limite_sup = coluna_med + (2 * coluna_std)
limite_inf = coluna_med - (2 * coluna_std)

index_outliers = []

for index, valor in coluna.items():
  if valor > limite_sup or valor < limite_inf:
    index_outliers.append(index)

len(index_outliers)

12641

Agora vamos dropar os outliers dos nossos dados e verificar a quantidade de linhas e colunas que sobraram.

In [17]:
df_clientes_cadastrados_sem_outliers = df_clientes_cadastrados_unicos.drop(index_outliers)
df_clientes_cadastrados_sem_outliers.shape

(425822, 18)

In [18]:
df_clientes_cadastrados_sem_outliers.describe()

,ID_Cliente,Tem_Carro,Tem_Casa_Propria,Qtd_Filhos,Rendimento_Anual,Idade,Tem_Celular,Tem_telefone_trabalho,Tem_telefone_fixo,Tem_email,Tamanho_Familia,Anos_empregado
count,4.258220e+05,425822.000000,425822.000000,425822.000000,425822.000000,425822.000000,425822.0,425822.000000,425822.000000,425822.000000,425822.000000,425822.000000
mean,6.021517e+06,0.364460,0.692435,0.425314,176384.728083,43.838707,1.0,0.206868,0.286134,0.104739,2.191683,5.744956
std,5.713306e+05,0.481279,0.461486,0.723577,75033.530551,11.525400,0.0,0.405061,0.451953,0.306217,0.896300,6.736649
min,5.008806e+06,0.000000,0.000000,0.000000,26100.000000,21.000000,1.0,0.000000,0.000000,0.000000,1.000000,-1.000000
25%,5.603204e+06,0.000000,0.000000,0.000000,117000.000000,34.000000,1.0,0.000000,0.000000,0.000000,2.000000,1.000000
50%,6.047739e+06,0.000000,1.000000,0.000000,157500.000000,43.000000,1.0,0.000000,0.000000,0.000000,2.000000,4.000000
75%,6.454044e+06,1.000000,1.000000,1.000000,225000.000000,53.000000,1.0,0.000000,1.000000,0.000000,3.000000,8.500000
max,7.999952e+06,1.000000,1.000000,19.000000,405000.000000,69.000000,1.0,1.000000,1.000000,1.000000,20.000000,48.000000


In [21]:
df_clientes_cadastrados_sem_outliers['Tem_Celular'].value_counts()

1    425822
Name: Tem_Celular, dtype: int64

Agora vamos dropar as colunas que não podemos utilizar para criar um modelo preditivo.

As colunas removidas serão as colunas `'Genero'` e `'Tem_Celular'`, dentro do nosso dataset  em todas as linhas os clientes tem celular, então não faria diferença em nosso modelo. Já a coluna 'Genero' é irrelevante e tem a possibilidade do modelo discriminar algum gênero, pois o importante é o individuo está adpto ou não a ter crédito ou não, já em um modelo da saúde o gênero pode ser relevante por causa da genética, mas para essa realidade não.

In [22]:
df_clientes_cadastrados_tratamento = df_clientes_cadastrados_sem_outliers.drop(['Genero', 'Tem_Celular'], axis=1)
df_clientes_cadastrados_tratamento.shape

(425822, 16)

***
#LIMPEZA DA BASE DE CLIENTES APROVADOS
***

Agora vamos limpar a base de Clientes Aprovados.

Primeiramente vamos verificar se cada linha do conjunto de dados é para um cliente. Como podemos perceber abaixo, cada cliente tem diversas linhas.


In [23]:
df_clientes_aprovados.head()

,ID_Cliente,Mes_referencia,Faixa_atraso
0,5001711,0,nenhum empréstimo
1,5001711,-1,1-29 dias
2,5001711,-2,1-29 dias
3,5001711,-3,1-29 dias
4,5001712,0,pagamento realizado


Utilizando o .info() vemos que não temos valores nulos nas colunas desse dataframe.


In [24]:
df_clientes_aprovados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   ID_Cliente      1048575 non-null  int64 
 1   Mes_referencia  1048575 non-null  int64 
 2   Faixa_atraso    1048575 non-null  object
dtypes: int64(2), object(1)
memory usage: 24.0+ MB


Utilizando o .value_counts() nos IDs, conseguimos verificar se possui Ids repetidos e como podemos ver tem ID que aparece 61 vezes, então precisamos pensar em uma maneira de deixar cada ID com apenas uma linha.

In [25]:
df_clientes_aprovados['ID_Cliente'].value_counts()

5145767    61
5116236    61
5037153    61
5002523    61
5028973    61
           ..
5149044     1
5079047     1
5137258     1
5023604     1
5023029     1
Name: ID_Cliente, Length: 45985, dtype: int64

Vamos olhar um exemplo de um ID que tem **muitos registros** utilizando o `.query()`

In [26]:
df_clientes_aprovados.query('ID_Cliente == 5001712')

,ID_Cliente,Mes_referencia,Faixa_atraso
4,5001712,0,pagamento realizado
5,5001712,-1,pagamento realizado
6,5001712,-2,pagamento realizado
7,5001712,-3,pagamento realizado
8,5001712,-4,pagamento realizado
9,5001712,-5,pagamento realizado
10,5001712,-6,pagamento realizado
11,5001712,-7,pagamento realizado
12,5001712,-8,pagamento realizado
13,5001712,-9,1-29 dias


Infelizmente não temos um padrão claro, podemos ver que tem clientes que tem 61 registros e outro 3 outros 5 e por aí vai.

In [27]:
df_clientes_aprovados.query('ID_Cliente == 5001711')

,ID_Cliente,Mes_referencia,Faixa_atraso
0,5001711,0,nenhum empréstimo
1,5001711,-1,1-29 dias
2,5001711,-2,1-29 dias
3,5001711,-3,1-29 dias


Vamos olhar as possíveis opções para a coluna 'Faixa_atraso', fazendo uma **contagem** com o `value_counts`, depois pegar o **index** e transformaçar em uma lista com o **to_list()**

In [28]:
df_clientes_aprovados.Faixa_atraso.value_counts().index.to_list() 

['pagamento realizado',
 '1-29 dias',
 'nenhum empréstimo',
 '30-59 dias',
 '>150 dias',
 '60-89 dias',
 '90-119 dias',
 '120-149 dias']

Vamos criar uma **coluna** para pegar o **mês de abertura** dos nossos dados, para isso vamos utilizar uma função **lambda** e percorrer todas as linhas existentes e pegar o valor **minimo** utilizando o **.min()**, indicando então, a abertura do crédito do cliente.

O `.apply` aplica uma função ao longo de um eixo do DataFrame. <br>

[Link da documentação]('https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html')

Uma função `lambda` é uma pequena função anônima. Uma função `lambda` pode receber qualquer número de argumentos, mas só pode ter uma expressão.


<img src="https://i1.faceprep.in/Companies-1/python-lambda-functions-new.png" alt="win" width="800"/>

In [29]:
df_registros_creditos_agrupados_ID = df_clientes_aprovados.groupby('ID_Cliente')
serie_abertura =  df_registros_creditos_agrupados_ID.apply(lambda x: min(x['Mes_referencia']))
serie_abertura.name = 'Abertura'
serie_abertura

ID_Cliente
5001711    -3
5001712   -18
5001713   -21
5001714   -14
5001715   -59
           ..
5150482   -28
5150483   -17
5150484   -12
5150485    -1
5150487   -29
Name: Abertura, Length: 45985, dtype: int64

In [30]:
df_clientes_aprovados = df_clientes_aprovados.merge(serie_abertura, on='ID_Cliente')
df_clientes_aprovados.head()

,ID_Cliente,Mes_referencia,Faixa_atraso,Abertura
0,5001711,0,nenhum empréstimo,-3
1,5001711,-1,1-29 dias,-3
2,5001711,-2,1-29 dias,-3
3,5001711,-3,1-29 dias,-3
4,5001712,0,pagamento realizado,-18


In [31]:
serie_final =  df_registros_creditos_agrupados_ID.apply(lambda x: max(x['Mes_referencia']))
serie_final.name = 'Final'
serie_final

ID_Cliente
5001711     0
5001712     0
5001713     0
5001714     0
5001715     0
           ..
5150482   -11
5150483     0
5150484     0
5150485     0
5150487     0
Name: Final, Length: 45985, dtype: int64

In [32]:
df_clientes_aprovados = df_clientes_aprovados.merge(serie_final, on='ID_Cliente')
df_clientes_aprovados.head()

,ID_Cliente,Mes_referencia,Faixa_atraso,Abertura,Final
0,5001711,0,nenhum empréstimo,-3,0
1,5001711,-1,1-29 dias,-3,0
2,5001711,-2,1-29 dias,-3,0
3,5001711,-3,1-29 dias,-3,0
4,5001712,0,pagamento realizado,-18,0


In [33]:
df_clientes_aprovados['Janela'] = df_clientes_aprovados['Final'] - df_clientes_aprovados['Abertura']
df_clientes_aprovados.head()

,ID_Cliente,Mes_referencia,Faixa_atraso,Abertura,Final,Janela
0,5001711,0,nenhum empréstimo,-3,0,3
1,5001711,-1,1-29 dias,-3,0,3
2,5001711,-2,1-29 dias,-3,0,3
3,5001711,-3,1-29 dias,-3,0,3
4,5001712,0,pagamento realizado,-18,0,18


In [34]:
df_clientes_aprovados['MOB'] = df_clientes_aprovados['Mes_referencia'] - df_clientes_aprovados['Abertura']
df_clientes_aprovados.head()

,ID_Cliente,Mes_referencia,Faixa_atraso,Abertura,Final,Janela,MOB
0,5001711,0,nenhum empréstimo,-3,0,3,3
1,5001711,-1,1-29 dias,-3,0,3,2
2,5001711,-2,1-29 dias,-3,0,3,1
3,5001711,-3,1-29 dias,-3,0,3,0
4,5001712,0,pagamento realizado,-18,0,18,18


Vamos transformar a coluna Faixa_atraso em uma coluna quantitativa, pois como no final iremos criar um modelo preditivo, então precisamos fazer essa transformação, para isso precisamos criar um dicionário e aplicar o  `.map()`

<img src="https://blog.fastformat.co/wp-content/uploads/2018/05/variaveisquantitativas_qualitativas.png" alt="win" width="800"/>

**Variáveis Quantitativas**<br> São representadas por meio de números resultantes de uma contagem ou mensuração. <br> Elas podem ser de dois tipos:
<br>
 - **Variáveis discretas**: os valores representam um conjunto finito ou enumerável de números, e que resultam de uma contagem, por exemplo: Número de filhos (0,1,2,…), número de bactérias por amostra, número de copos de cerveja tomados por dia.<br>
 - **Variáveis contínuas**: os valores pertencem a um intervalo de números reais e representam uma mensuração como por exemplo altura ou peso de uma pessoa. Nesses casos números fracionais fazem sentido. Exemplo: tempo (relógio) e pressão arterial.<br>

**Variáveis Qualitativas**<br> Representam uma qualidade (ou atributo) de um indivíduo pesquisado, são definidas por várias categorias. São características que não possuem valores quantitativos. <br>Essas variáveis podem ser de dois tipos:<br>
- **Variável nominal**: quando não existe nenhuma ordenação nas possíveis representações. Exemplos: sexo, cor dos olhos, cor do cabelo, fumante/não fumante.
- **Variável ordinal**: quando apresentam uma ordem nos seus resultados. Exemplos: escolaridade (1, 2, 3 graus), mês de observação (janeiro, fevereiro, …, dezembro.)

In [35]:
dict_faixa_atraso_ind ={'nenhum empréstimo': 0, 'pagamento realizado': 1,
                        '1-29 dias': 2, '30-59 dias': 3, '60-89 dias': 4,
                        '90-119 dias': 5, '120-149 dias': 6, '>150 dias': 7}

`.map()` Mapeia os valores das séries de acordo com a correspondência de entrada.

Usado para substituir cada valor em uma Série por outro valor, que pode ser derivado de uma função, um dicionário ou uma Série.<br>[Link da Documentação]('https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html')


In [36]:
df_clientes_aprovados['Ind_Faixa_atraso'] = df_clientes_aprovados['Faixa_atraso'].map(dict_faixa_atraso_ind)
df_clientes_aprovados.head()

,ID_Cliente,Mes_referencia,Faixa_atraso,Abertura,Final,Janela,MOB,Ind_Faixa_atraso
0,5001711,0,nenhum empréstimo,-3,0,3,3,0
1,5001711,-1,1-29 dias,-3,0,3,2,2
2,5001711,-2,1-29 dias,-3,0,3,1,2
3,5001711,-3,1-29 dias,-3,0,3,0,2
4,5001712,0,pagamento realizado,-18,0,18,18,1


Para criar um modelo preditivo, precisamos de uma coluna que possui os dados que eu queremos prever, essa variável é nomeada como variável target e as demais variáveis são chamadas de variáveis preditoras, pois vão auxiliar na predição do resultado.

Portanto, precisamos identificar quais são os clientes que são pagadores ruins, pois não queremos clientes que não paguem os emprestimos que iremos conceder, mas por outro lado queremos clientes que atrasem suas dividas, tendo em vista que ganhamos dinheiro com o juros das dívidas atrasadas também. 

Portanto precisamos decidir até qual janela será um bom pagador e qual será um mau pagador para nós do TowerBank. 

Por enquanto, vamos escolher até 89 dias de atraso é um bom pagador dos seus créditos em aberto, já os acima serão considerados um maus pagadores. 

Para realizar essa classificação, precisamos utilizar o método `.apply()` novamente e passar uma logica em cada linha do dataframe e verificar se o indice de atraso é **maior que 3**, pois segundo o dicionário que criados a chave 3 é o valor **'60-89 dias'** e criar a nova coluna chamada **'Mau'**.

In [37]:
df_clientes_aprovados['Mau'] = df_clientes_aprovados.apply(lambda x: 1 if x['Ind_Faixa_atraso'] > 3 else 0, axis=1)
df_clientes_aprovados.head()

,ID_Cliente,Mes_referencia,Faixa_atraso,Abertura,Final,Janela,MOB,Ind_Faixa_atraso,Mau
0,5001711,0,nenhum empréstimo,-3,0,3,3,0,0
1,5001711,-1,1-29 dias,-3,0,3,2,2,0
2,5001711,-2,1-29 dias,-3,0,3,1,2,0
3,5001711,-3,1-29 dias,-3,0,3,0,2,0
4,5001712,0,pagamento realizado,-18,0,18,18,1,0


Bom, como podemos ver acima, temos várias linhas por ID, então precisamos deixar apenas uma linha para cada ID e como faremos isso?

Primeiramente vamos apenas pegar as colunas **ID_Cliente, Abertura, Final e Janela** e agrupar pelo ID utilizando o `.groupBy()`   [Link da Documentação](https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.groupby.html)

Depois vamos vamos utilizar o `.apply` para aplicar uma lógica em cada linha do dataframe utilizando uma função lambda e pegando o primeiro regristro do ID usando o `.iloc[0]` ([Link da Documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html)), pois essas colunas tem o mesmo valor sempre, então podemos pegar qualquer uma.

Por fim, vamos utilizar o `.reset_index()` ([Link da Documentação](https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.reset_index.html)) para tirar o ID do index e voltar para coluna e depois salvar  em uma variável.

In [38]:
df_registros_creditos_ID = df_clientes_aprovados[['ID_Cliente','Abertura', 'Final', 'Janela']].groupby('ID_Cliente').apply(lambda x: x.iloc[0]).reset_index(drop=True)
df_registros_creditos_ID

,ID_Cliente,Abertura,Final,Janela
0,5001711,-3,0,3
1,5001712,-18,0,18
2,5001713,-21,0,21
3,5001714,-14,0,14
4,5001715,-59,0,59
...,...,...,...,...
45980,5150482,-28,-11,17
45981,5150483,-17,0,17
45982,5150484,-12,0,12
45983,5150485,-1,0,1


Vamos pegar apenas os clientes que estão mais de uma ano na base de dados, pois temos mais estabilidade na base de dados.

In [39]:
df_clientes_aprovados_tratamento1 = df_clientes_aprovados.query('Janela >= 12').copy()
df_clientes_aprovados_tratamento1.head()

,ID_Cliente,Mes_referencia,Faixa_atraso,Abertura,Final,Janela,MOB,Ind_Faixa_atraso,Mau
4,5001712,0,pagamento realizado,-18,0,18,18,1,0
5,5001712,-1,pagamento realizado,-18,0,18,17,1,0
6,5001712,-2,pagamento realizado,-18,0,18,16,1,0
7,5001712,-3,pagamento realizado,-18,0,18,15,1,0
8,5001712,-4,pagamento realizado,-18,0,18,14,1,0


Vamos agrupar os clientes por ID e vamos utilizar o método apply para aplicar uma logica para cada ID, essa logica é definida na função verifica.

Essa função recebe os registros de cada cliente, salva todas as faixas de atraso na variavel lista_status.

Depois utilizamos o if para verificar se as faixas de atraso daquele ID contem alguma das faixas de atrasos determinadas, caso tenha retorna 1 e caso não retorna 0

Depois de aplicar em todos os ID, vamos utilizar o reset_index para ter a coluna de ID.

No final temos o DataFrame df_registros_creditos_id, com a coluna ID e a coluna Mau.

In [40]:
def verifica(registros):
  lista_status = registros['Faixa_atraso'].to_list()
  if '60-89 dias' in lista_status or '90-119 dias' in lista_status or '120-149 dias' in lista_status or '>150 dias' in lista_status:
    return 1
  else:
    return 0
  
df_registros_creditos_id = pd.DataFrame(df_clientes_aprovados_tratamento1.groupby('ID_Cliente').apply(verifica)).reset_index()
df_registros_creditos_id.columns = ['ID_Cliente', 'Mau']
df_registros_creditos_id.head()

,ID_Cliente,Mau
0,5001712,0
1,5001713,0
2,5001714,0
3,5001715,0
4,5001717,0


#Criação da Base para predição

Vamos pegar as duas bases (Clientes Aprovados e Clientes cadastrados) e juntar elas utilizando os .merge e ter a base para a criação do modelo preditivo.

In [41]:
df_registro_clientes_targets = df_clientes_cadastrados_tratamento.merge(df_registros_creditos_id, on='ID_Cliente')
df_registro_clientes_targets.head()

,ID_Cliente,Tem_Carro,Tem_Casa_Propria,Qtd_Filhos,Rendimento_Anual,Categoria_de_renda,Grau_Escolaridade,Estado_Civil,Moradia,Idade,Tem_telefone_trabalho,Tem_telefone_fixo,Tem_email,Ocupacao,Tamanho_Familia,Anos_empregado,Mau
0,5008806,1,1,0,112500.0,Empregado,Ensino medio,Casado,Casa propria,59.0,0,0,0,Seguranca,2.0,3.1,0
1,5008810,0,1,0,270000.0,Associado comercial,Ensino medio,Solteiro,Casa propria,52.0,0,1,1,Venda,1.0,8.4,0
2,5008811,0,1,0,270000.0,Associado comercial,Ensino medio,Solteiro,Casa propria,52.0,0,1,1,Venda,1.0,8.4,0
3,5008812,0,1,0,283500.0,Pensionista,Ensino superior,Separado,Casa propria,62.0,0,0,0,Outros,1.0,-1.0,0
4,5008813,0,1,0,283500.0,Pensionista,Ensino superior,Separado,Casa propria,62.0,0,0,0,Outros,1.0,-1.0,0


Agora transformar os dados em **CSV**  utilizando o **.to_csv()** removendo os index utilizando o parâmetro **index = False** e baixar com o aquivo com uma biblioteca do **Google Colab**.

In [42]:
df_registro_clientes_targets.to_csv('dados_clientes_targets.csv',index= False)
#files.download('dados_clientes_targets.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>